In [ ]:
"""
crew: team of agents
 |
agent: member with one specific role
"""

In [1]:
import os
from dotenv import load_dotenv
from crewai import Crew, Agent, Task
from crewai_tools import SerperDevTool, ScrapeWebsiteTool, YoutubeChannelSearchTool
from pydantic import BaseModel
from typing import List
from crewai_tools import tool
import yfinance as yf
from crewai.process import Process
from langchain_openai import ChatOpenAI

load_dotenv()

os.environ["OPENAI_MODEL_NAME"] = "gpt-4o"

In [ ]:
international_chef = Agent(
    role="International Chef",
    goal="Create ethnic cuisine recipies that are easy to cook at home",
    backstory="""
    You are an famous chef that specializes in couisine from countries all around the world.
    You know how to cook the most traditional dishes from all cultures but you also know how to adapt them for people to be able to cook them at home.
    """,
    verbose=True,
    allow_delegation=False, # do not ask another agent to help, do task alone
)

healthy_chef = Agent(
    role="Healthy Chef",
    goal="Turn any recipe into a healthy vegetarian recipe that is easy to cook with home ingredients.",
    backstory="""
    You are a chef specialized in healthy cooking.
    You can take any recipe and change the ingredients to make it vegetarian friendly without loosing the escense of the dish and what makes it delicious.
    """,
    verbose=True,
    allow_delegation=False,
)

normal_recipe = Task(
    description="Come up with a {dish} that serves {people} people.",
    agent=international_chef,
    expected_output="Your answer MUST have three sections, the ingredients required with their quantities, the preparation instructions and serving suggestions", # important for better performance
    output_file="normal_recipe.md",
)

healthy_recipe = Task(
    description="Replace the ingredients of a recipe to make it vegetarian without making it less delicious, adjust if needed.",
    agent=healthy_chef,
    expected_output="Your answer MUST have four sections, the ingredients required with their quantities, the preparation instructions, serving suggestions and an explanation of the replaced ingredients.",
    output_file="healthy_recipe.md",
)

crew = Crew(
    tasks=[ # sequence is important
        normal_recipe,
        healthy_recipe,
    ],
    agents=[
        international_chef,
        healthy_chef,
    ],
    verbose=True,
)

result = crew.kickoff(
    inputs=dict(
        dish="Greek dinner",
        people="5",
    )
)

In [ ]:
from regex import B


class SubSection(BaseModel):
    title: str
    content: str

class BlogPost(BaseModel):
    title: str
    introduction: str
    sections: List[SubSection]
    sources: List[str]
    hashtags: List[str]

search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()

researcher = Agent(
    role="Senior Researcher",
    goal="Search the web, extract and analyze information.",
    backstory="""
    You produce the highest quality research possible.
    You use multiple sources of information and you always double check your sources to make sure they are true and up to date.
    You want to impress your coworkers with your work.
    """, # performs better when agent wants to impress someone or gets tips
    allow_delegation=False,
    tools=[
        search_tool,
        scrape_tool,
    ],
    max_iter=10, # gardrale to avoid infinite loops
    verbose=True
)

editor = Agent(
    role="Senior Writer/Editor",
    goal="Write engaging blog posts.",
    backstory="""
    You write content that keeps people engaged and entertained.
    Your content is easy to read it is informative and it makes people want to share it with their friends.
    You are working for a very important client.
    """, 
    verbose=True
)

task = Task(
    description="Write a blog post about {topic}",
    agent=editor,
    expected_output="A blog post with an introduction, at least three sub-sections of content, links to sources, a set of suggested hashtags for social media and a catchy title.",
    output_file="blog_post.md",
    output_json=BlogPost # output_pydatic=BlogPost # force the structured output to use in DB or API or UI
)

crew = Crew(
    tasks=[task],
    agents=[
        researcher,
        editor,
    ],
    verbose=True,
)

result = crew.kickoff(
    inputs=dict(
        topic="The biggest box office flops of 2024",
    ),
)

In [ ]:
search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()
youtube_tool = YoutubeChannelSearchTool()

researcher = Agent(
    role="Senior Researcher",
    goal="Search the web, extract and analyze information.",
    backstory="""
    You produce the highest quality research possible.
    You use multiple sources of information and you always double check your sources to make sure they are true and up to date.
    You want to impress your coworkers with your work.
    """,
    allow_delegation=False,
    tools=[
        search_tool,
        scrape_tool,
        youtube_tool,
    ],
    max_iter=10,
    verbose=True,
)

marketer = Agent(
    role="Senior Marketer",
    goal="Come up with ideas that generate viral and useful content.",
    backstory="""
    You work at a marketing agency.
    You are the best at coming up with ideas to make content go viral.
    Your ideas are used for video, advertising, social media marketing, the content you produce appeals to a young audience.
    """,
    verbose=True,
)

writer = Agent(
    role="Senior Writer",
    goal="Write scripts for viral Youtube videos.",
    backstory="""
    You write scripts for videos that keep people engaged and entertained.
    Your content is easy and fun to watch, it is informative and it makes people want to share it with their friends.
    You are working for a very important client.
    """,
    verbose=True,
)

"""
brainstorm(marketor) -> selection(writer) -> research(researcher)    -> script(writer)
                                          -> competitors(researcher)
                                          -> inspiration(researcher)
"""

brainstorm_task = Task(
    description="Come up with 5 video ideas for a Youtube channel in the {industry} industry",
    agent=marketer,
    expected_output="Your answer MUST be a list of 5 ideas for a Youtube video with an explanation of what the angle of the video would be.",
    output_file="ideas_task.md",
    human_input=True, # human needs to approve in case human doesn't like the outcome
)

selection_task = Task(
    description="Select a video idea that has the highest potential of going viral.",
    agent=writer,
    expected_output="Your answer MUST include the idea that was selected as well as an explanation of why that selection was made.",
    human_input=True,
    context=[
        brainstorm_task, # wait until brainstorm finishes and receive the outputs
    ],
    output_file="selection_task.md",
)

research_task = Task(
    description="Do all the research required to write the script of a medium length video about the selected idea.",
    agent=researcher,
    expected_output="You answer must have all the information a writer would need to write a Youtube script.",
    async_execution=True, # run at the same time
    context=[
        selection_task,
    ],
    output_file="research_task.md",
)

competitors_task = Task(
    description="Search for videos or articles in the {industry} industry that are similar to the video idea we are working on and suggest ways our video can be different from theirs.",
    expected_output="Your answer must have a list of suggestions writers can follow to make sure the video is as unique and as different from competitors as possible.",
    agent=researcher,
    async_execution=True,
    context=[
        selection_task,
    ],
    output_file="competitors_task.md",
)

inspiration_task = Task(
    description="Search for videos or articles that are similar to the video idea we are working on but from other industries.",
    expected_output="Your answer must have a list of examples of articles and videos that have a similar angle as the video we are making but that are in different industries.",
    agent=researcher,
    async_execution=True,
    context=[
        selection_task,
    ],
    output_file="inspiration_task.md",
)

script_task = Task(
    description="Write the script for a Youtube video for a channel in the {industry} industry.",
    expected_output="A script for a Youtube video with a title, an introduction, at least three sections, and an outro. Make sure to also include the prompt to generate a thumbnail for the video.",
    agent=writer,
    context=[
        selection_task,
        research_task,
        competitors_task,
        inspiration_task,
    ],
    output_file="script_task.md",
)

crew = Crew(
    agents=[
        researcher,
        writer,
        marketer,
    ],
    tasks=[
        brainstorm_task,
        selection_task,
        research_task,
        inspiration_task,
        competitors_task,
        script_task,
    ],
    verbose=2,
)

result = crew.kickoff(
    inputs=dict(
        industry="Hot Sauce",
    ),
)

In [2]:
from regex import T


class Tools:
    @tool("One month stock price history") # describe the tool
    def stock_price(ticker):
        """
        Useful to get a month's worth of stock price data as CSV.
        The input of this tool should a ticker, for example AAPL, NET, TSLA etc...
        """
        stock = yf.Ticker(ticker)
        return stock.history(period="1mo").to_csv()

    @tool("Stock news URLs")
    def stock_news(ticker):
        """
        Useful to get URLs of news articles related to a stock.
        The input to this tool should be a ticker, for example AAPL, NET
        """
        stock = yf.Ticker(ticker)
        return list(map(lambda x: x["link"], stock.news))

    @tool("Company's income statement")
    def income_stmt(ticker):
        """
        Useful to get the income statement of a stock as CSV.
        The input to this tool should be a ticker, for example AAPL, NET
        """
        stock = yf.Ticker(ticker)
        return stock.income_stmt.to_csv()

    @tool("Balance sheet")
    def balance_sheet(ticker):
        """
        Useful to get the balance sheet of a stock as CSV.
        The input to this tool should be a ticker, for example AAPL, NET
        """
        stock = yf.Ticker(ticker)
        return stock.balance_sheet.to_csv()

    @tool("Get insider transactions")
    def insider_transactions(ticker):
        """
        Useful to get insider transactions of a stock as CSV.
        The input to this tool should be a ticker, for example AAPL, NET
        """
        stock = yf.Ticker(ticker)
        return stock.insider_transactions.to_csv()

class Agents:
    def technical_analyst(self):
        return Agent(
            role="Technical Analyst",
            goal="Analyses the movements of a stock and provides insights on trends, entry points, resistance and support levels.",
            backstory="An expert in technical analysis, you're known for your ability to predict stock movements and trends based on historical data. You provide valuable insights to your customers.",
            verbose=True,
            tools=[
                Tools.stock_price,
            ],
        )

    def researcher(self):
        return Agent(
            role="Researcher",
            goal="Gathers, interprets and summarizes vasts amounts of data to provide a comprehensive overview of the sentiment and news surrounding a stock.",
            backstory="You're skilled in gathering and interpreting data from various sources to give a complete picture of a stock's sentiment and news. You read each data source carefuly and extract the most important information. Your insights are crucial for making informed investment decisions.",
            verbose=True,
            tools=[
                Tools.stock_news,
                SerperDevTool(), # search the web
                ScrapeWebsiteTool(), # read the web
            ],
        )

    def financial_analyst(self):
        return Agent(
            role="Financial Analyst",
            goal="Uses financial statements, insider trading data, and other financial metrics to evaluate a stock's financial health and performance.",
            backstory="You're a very experienced investment advisor who uses a combination of technical and fundamental analysis to provide strategic investment advice to your clients. You look at a company's financial health, market sentiment, and qualitative data to make informed recommendations.",
            verbose=True,
            tools=[
                Tools.balance_sheet,
                Tools.income_stmt,
                Tools.insider_transactions,
            ],
        )

    def hedge_fund_manager(self):
        return Agent(
            role="Hedge Fund Manager",
            goal="Manages a portfolio of stocks and makes strategic investment decisions to maximize returns using insights from financial analysts, technical analysts, and researchers.",
            backstory="You're a seasoned hedge fund manager with a proven track record of making profitable investment decisions. You're known for your ability to manage risk and maximize returns for your clients.",
            verbose=True,
        )
    
class Tasks:
    def research(self, agent):
        return Task(
            description="Gather and analyze the latest news and market sentiment surrounding the stock of {company}. Provide a summary of the news and any notable shifts in market sentiment.",
            expected_output=f"Your final answer MUST be a detailed summary of the news and market sentiment surrounding the stock. Include any notable shifts in market sentiment and provide insights on how these factors could impact the stock's performance.",
            agent=agent,
            output_file="stock_news.md",
        )

    def technical_analysis(self, agent):
        return Task(
            description="Conduct a detailed technical analysis of the price movements of {company}'s stock and trends identify key support and resistance levels, chart patterns, and other technical indicators that could influence the stock's future performance. Use historical price data and technical analysis tools to provide insights on potential entry points and price targets.",
            expected_output=f"Your final answer MUST be a detailed technical analysis report that includes key support and resistance levels, chart patterns, and technical indicators. Provide insights on potential entry points, price targets, and any other relevant information that could help your customer make informed investment decisions.",
            agent=agent,
            output_file="technical_analysis.md",
        )

    def finacial_analysis(self, agent):
        return Task(
            description="Analyze {company}'s financial statements, insider trading data, and other financial metrics to evaluate the stock's financial health and performance. Provide insights on the company's revenue, earnings, cash flow, and other key financial metrics. Use financial analysis tools and models to assess the stock's valuation and growth potential.",
            expected_output=f"Your final answer MUST be a detailed financial analysis report that includes insights on the company's financial health, performance, and valuation. Provide an overview of the company's revenue, earnings, cash flow, and other key financial metrics. Use financial analysis tools and models to assess the stock's valuation and growth potential.",
            agent=agent,
            output_file="financial_analysis.md",
        )

    def investment_recommendation(self, agent, context):
        return Task(
            description="Based on the research, technical analysis, and financial analysis reports, provide a detailed investment recommendation for {company}'s stock. Include your analysis of the stock's potential risks and rewards, and provide a clear rationale for your recommendation.",
            expected_output=f"Your final answer MUST be a detailed investment recommendation report to BUY or SELL the stock that includes your analysis of the stock's potential risks and rewards. Provide a clear rationale for your recommendation based on the research, technical analysis, and financial analysis reports.",
            agent=agent,
            context=context,
            output_file="investment_recommendation.md",
        )

agents = Agents()
tasks = Tasks()

researcher = agents.researcher()
technical_analyst = agents.technical_analyst()
financial_analyst = agents.financial_analyst()
hedge_fund_manager = agents.hedge_fund_manager()

research_task = tasks.research(researcher)
technical_task = tasks.technical_analysis(technical_analyst)
financial_task = tasks.finacial_analysis(financial_analyst)
recommend_task = tasks.investment_recommendation(
    hedge_fund_manager,
    [
        research_task,
        technical_task,
        financial_task,
    ],
)

crew = Crew(
    agents=[
        researcher,
        technical_analyst,
        financial_analyst,
        hedge_fund_manager,
    ],
    tasks=[
        research_task,
        technical_task,
        financial_task,
        recommend_task,
    ],
    verbose=True,
    process=Process.hierarchical, # create manager to oversee the crew # default: Process.sequential 
    manager_llm=ChatOpenAI(model="gpt-4o"),
    memory=True, # use memory and database is created
)

result = crew.kickoff(
    inputs=dict(
        company="Salesforce",
    ),
)

 [DEBUG]: Working Agent: Crew Manager
 [INFO]: Starting Task: Gather and analyze the latest news and market sentiment surrounding the stock of Salesforce. Provide a summary of the news and any notable shifts in market sentiment.


> Entering new CrewAgentExecutor chain...
To gather and analyze the latest news and market sentiment surrounding the stock of Salesforce, I need to start by collecting the most recent news articles and data. The Researcher can help with gathering news information, while the Technical Analyst can analyze market sentiment. I will begin by delegating the task of collecting the latest news on Salesforce to the Researcher.

Action: Delegate work to co-worker
Action Input: {"task": "Collect the latest news articles and reports on Salesforce stock.", "context": "We need to gather the most recent news and developments related to Salesforce to understand the current market sentiment.", "coworker": "Researcher"}


> Entering new CrewAgentExecutor chain...
To gather the

2025-05-24 04:56:07,222 - 8523810880 - base.py-base:126 - ERROR: Failed to get ticker 'CRM' reason: Expecting value: line 1 column 1 (char 0)
2025-05-24 04:56:08,234 - 8523810880 - history.py-history:210 - ERROR: $CRM: possibly delisted; No price data found  (period=1mo)


 

Date,Open,High,Low,Close,Adj Close,Volume


I'm unable to fetch or analyze real-time data or stock price history directly. However, you can use the CSV data you've just retrieved to perform a technical analysis. Here's a general approach you can take to analyze Salesforce's stock movements:

1. **Trend Analysis**: Look at the stock price over the past month to determine if it's in an uptrend, downtrend, or sideways trend. An uptrend would be indicated by higher highs and higher lows, a downtrend by lower highs and lower lows, and a sideways trend by a lack of clear direction.

2. **Support and Resistance Levels**: Identify key support and resistance levels. Support is a price level where the stock tends to stop falling and bounce back up, while resistance is where it tends to stop rising and pull back.

3. **Moving Averages**: Calculate moving averages (e.g., 10-day, 20-day) to smooth out price data and identify trends. If the stock price is above the moving average, it could indica

2025-05-24 04:57:01,517 - 8523810880 - holders.py-holders:84 - ERROR: 429 Client Error: Too Many Requests for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary//CRM?modules=institutionOwnership%2CfundOwnership%2CmajorDirectHolders%2CmajorHoldersBreakdown%2CinsiderTransactions%2CinsiderHolders%2CnetSharePurchaseActivity&corsDomain=finance.yahoo.com&formatted=false&crumb=Edge%3A+Too+Many+Requests


 

""




> Finished chain.
 

Agent stopped due to iteration limit or time limit.

It seems we reached a limit before completing the task. Let's continue by gathering more detailed insights regarding the financial statements, insider trading data, and other financial metrics necessary for a comprehensive financial analysis of Salesforce. 

Since the previous action wasn't completed, I'll ask the Financial Analyst again to provide the analysis of Salesforce's financial statements. Additionally, I'll gather insights from the Researcher on insider trading data.

Thought: It's essential to gather insights on Salesforce's financial statements and insider trading data for a comprehensive evaluation of the company's financial health and performance.

Action: Ask question to co-worker
Action Input: {"question": "Please provide a detailed analysis of Salesforce's financial statements, focusing on revenue, earnings, and cash flow. Include any trends or insights from recent quarters that might a